In [ ]:
#Cell_1.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.preprocessing import scale
from sklearn.metrics import mean_squared_error

import warnings
warnings.simplefilter('ignore')

import plotly

plt.rcParams['font.family'] = 'Yu Mincho' # for Japanese in graph (Win)

In [ ]:
#Cell_2.
from greykite.algo.changepoint.adalasso.changepoint_detector import ChangepointDetector
from greykite.algo.forecast.silverkite.constants.silverkite_column import SilverkiteColumn
from greykite.common.features.timeseries_features import get_available_holidays_across_countries
from greykite.framework.input.univariate_time_series import UnivariateTimeSeries
from greykite.framework.templates.autogen.forecast_config import ForecastConfig
from greykite.framework.templates.autogen.forecast_config import MetadataParam
from greykite.framework.templates.autogen.forecast_config import ModelComponentsParam
from greykite.framework.templates.forecaster import Forecaster
from greykite.framework.templates.model_templates import ModelTemplateEnum
from greykite.framework.utils.result_summary import summarize_grid_search_results

In [ ]:
#Cell_3.
csv_in="yields.csv"
df=pd.read_csv(csv_in,skiprows=0,header=0,delimiter=',',encoding="shift-jis")
print(df.shape) #確認
print(df.head()) #確認
print(type(df['time'][0]))

In [ ]:
#Cell_4.
from datetime import datetime
for i in range(len(df)):
    df.loc[i,'time']= datetime.fromtimestamp((df.loc[i,'time']/1000))

In [ ]:
#Cell_5.
display(df.head())
display(df.tail())

In [ ]:
#Cell_6.
df_sorted = df.sort_values(by='time')
display(df_sorted.tail(20))

In [ ]:
#Cell_7.
df_sorted['time']=pd.to_datetime(df_sorted['time'])
df_sorted['time']=df_sorted['time'].dt.date

df_sorted=df_sorted.reset_index(drop=True)
df_sorted['time']=pd.to_datetime(df_sorted['time'])
display(df_sorted.head())

In [ ]:
#Cell_8.
csv_in2="8316_2018.csv"
df2=pd.read_csv(csv_in2,skiprows=1,header=0,delimiter=',',encoding="shift-jis")
print(df2.shape) #確認
print(df2.head()) #確認
print(type(df2['日付'][0]))
df2['日付']=pd.to_datetime(df2['日付'])
print(type(df2['日付'][0]))

In [ ]:
#Cell_9.
df2.columns=['time','start','high','low','close','volume','adj-close']
display(df2.head())

In [ ]:
#Cell_10.
df_merged=pd.merge(df2,df_sorted,on='time',how='left')
display(df_merged.head())

In [ ]:
#Cell_11.
df_merged2=df_merged[['time','close','JP01','JP02','JP05','JP10','US01','US02','US05','US10']]

print(df_merged2.shape)
print(df_merged2.info())
display(df_merged2.head())

In [ ]:
#Cell_12.
df2=df_merged2.fillna(method="ffill")

In [ ]:
#Cell_13.
metadata = MetadataParam(
    time_col="time", # 日付の列名
    value_col="close", # 値の列名
    freq="D", # 日付の単位
    train_end_date=pd.datetime(2018, 5, 20)
)

In [ ]:
#Cell_14.
regressors=dict(
    regressor_cols=[
        ['JP01','JP02','JP05','JP10','US01','US02','US05','US10']
    ]
)

In [ ]:
#Cell_15.
yearly_seasonality_order = 10
weekly_seasonality_order = 5
seasonality = {
    "yearly_seasonality": yearly_seasonality_order,
    "quarterly_seasonality": False,
    "monthly_seasonality": False,
    "weekly_seasonality": weekly_seasonality_order,
    "daily_seasonality": False
}

In [ ]:
#Cell_16.
growth = {
 "growth_term": "linear"
}
changepoints = {
 "changepoints_dict": dict(
     method="auto",
     yearly_seasonality_order=10,
     regularization_strength=0.5,
     resample_freq="7D",
     potential_changepoint_n=25,
     yearly_seasonality_change_freq="365D",
     no_changepoint_distance_from_end="365D"
 )
}

In [ ]:
#Cell_17.
model_components = ModelComponentsParam(
    seasonality=seasonality,
    growth=growth,
    #events=events,
    #changepoints=changepoints,
    autoregression=None,
    regressors=regressors,
    uncertainty={
     "uncertainty_dict": "auto",
    },
    custom={
     "fit_algorithm_dict": {
         "fit_algorithm": "ridge",
     },
     #"extra_pred_cols": extra_pred_cols
    }
)

In [ ]:
#Cell_18.
forecaster = Forecaster()
result = forecaster.run_forecast_config(
    df=df2,
    config=ForecastConfig(
        model_template=ModelTemplateEnum.SILVERKITE.name,
        forecast_horizon=14,
        coverage=0.95, # 予測区間を95%に設定
        metadata_param=metadata,
        model_components_param=model_components
    )
)

In [ ]:
#Cell_19.
print(result.model[-1].summary(max_colwidth=50))

In [ ]:
#Cell_20.
backtest = result.backtest
fig = backtest.plot()
plotly.io.show(fig)

In [ ]:
#Cell_21.
fig = result.forecast.plot_components()
plotly.io.show(fig)

In [ ]:
#Cell_22.
model = ChangepointDetector()
res = model.find_trend_changepoints(
    df=df2, # データフレーム
    time_col="time", # 日付の列名
    value_col="close", # 値の列名
    yearly_seasonality_order=0, # 年周期を表現するためのフーリエ級数の次数(今回は年周期を仮定してないので0)
    regularization_strength=0.5, # adaptive lassoの罰則の強さ(0~1の値)
    resample_freq="7D", # データを括る粒度
    potential_changepoint_n=20, # 変化点の数、値を大きくすると計算負荷が高くなる
    yearly_seasonality_change_freq=None, # 年周期の値が変わる間隔(今回は年周期ないので無視)
    no_changepoint_distance_from_end="14D", # ラスト14日は変化点なし
    actual_changepoint_min_distance="7D" # 変化点どうしの感覚は最低7日空ける
)

fig = model.plot(
    observation=True,
    trend_estimate=False,
    trend_change=True,
    yearly_seasonality_estimate=False,
    adaptive_lasso_estimate=True,
    plot=False)
plotly.io.show(fig)

In [ ]:
#Lets try a customized model.
#Cell_23.
growth = {
    "growth_term": "linear" # 線形トレンド
}

changepoints = {
    "changepoints_dict": {
        "method": "auto",
        "yearly_seasonality_order": 0,
        "regularization_strength": 0.5,
        "resample_freq": "7D",
        "potential_changepoint_n": 20,
        "yearly_seasonality_change_freq": None,
        "no_changepoint_distance_from_end": "14D",
        "actual_changepoint_min_distance": "7D"
    }
}

seasonality = {
    "yearly_seasonality": False,
    "quarterly_seasonality": False,
    "monthly_seasonality": False,
    "weekly_seasonality": 2, # 週周期の次数
    "daily_seasonality": False
}

events = {
    "holiday_lookup_countries": None # 祝日なし
}

model_components = ModelComponentsParam(
    seasonality=seasonality,
    growth=growth,
    changepoints=changepoints,
    autoregression={
        "autoreg_dict": None # 自己回帰の項なし
    },
    events=events,
    regressors=regressors,
    lagged_regressors={
        "lagged_regressor_dict": None # 説明変数のラグも当然なし
    },
    custom={
        "fit_algorithm_dict": {
            "fit_algorithm": "ridge"
        },
        "feature_sets_enabled": {
            SilverkiteColumn.COLS_DAY_OF_WEEK: False, # 曜日ダミーの有無
            SilverkiteColumn.COLS_TREND_WEEKEND: False, # 平日/週末とトレンドとの交互作用の有無
            SilverkiteColumn.COLS_TREND_DAY_OF_WEEK: False, # 曜日とトレンドとの交互作用の有無
            SilverkiteColumn.COLS_TREND_WEEKLY_SEAS: False # 週周期の変化を許容するかどうか
        }
    },
    uncertainty={
        "uncertainty_dict": {
            "uncertainty_method": "simple_conditional_residuals",
            "params": {
                "quantiles": [0.025, 0.975],
                "conditional_cols": ["month"], # 予測区間を月ごとに計算する
                "quantile_estimation_method": "normal_fit"
            }

        },
    }
)

In [ ]:
#Cell_24.
custom_result = forecaster.run_forecast_config(
    df=df2,
    config=ForecastConfig(
        model_template=ModelTemplateEnum.SILVERKITE.name,
        forecast_horizon=14,
        coverage=0.95,  # 95% prediction intervals
        model_components_param=model_components,
        metadata_param=metadata
    )
)

In [ ]:
#Cell_25.
backtest = custom_result.backtest
fig = backtest.plot()
plotly.io.show(fig)

In [ ]:
#Cell_26.
print(custom_result.model[-1].summary(max_colwidth=50))

In [ ]:
#Cell_27.
fig = custom_result.forecast.plot_components()
plotly.io.show(fig)

In [ ]:
#Cell_28.
custom_result.backtest.test_evaluation